In [6]:
%idle_timeout 2880
%glue_version 3.0
%worker_type G.1X
%number_of_workers 2
%%configure 
{
  "--conf": "spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions",
  "--datalake-formats": "iceberg"
}

Welcome to the Glue Interactive Sessions Kernel
For more information on available magic commands, please type %help in any new cell.

Please view our Getting Started page to access the most up-to-date information on the Interactive Sessions kernel: https://docs.aws.amazon.com/glue/latest/dg/interactive-sessions.html
Installed kernel version: 1.0.2 
Current idle_timeout is None minutes.
idle_timeout has been set to 2880 minutes.
Setting Glue version to: 3.0
Previous worker type: None
Setting new worker type to: G.1X
Previous number of workers: None
Setting new number of workers to: 2
The following configurations have been updated: {'--conf': 'spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions', '--datalake-formats': 'iceberg'}


### create the variables

In [1]:
catalog_name = "glue_catalog"
bucket_name = "andres-lagos-bucket"
bucket_prefix = "iceberg"
database_name = "dictionary_quality"
warehouse_path = f"s3://{bucket_name}/{bucket_prefix}/"

Trying to create a Glue session for the kernel.
Session Type: glueetl
Worker Type: G.1X
Number of Workers: 2
Session ID: ee908911-7283-407c-808d-55dff5637e9e
Applying the following default arguments:
--glue_kernel_version 1.0.2
--enable-glue-datacatalog true
--conf spark.sql.extensions=org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions
--datalake-formats iceberg
Waiting for session ee908911-7283-407c-808d-55dff5637e9e to get into ready status...
Session ee908911-7283-407c-808d-55dff5637e9e has been created.



In [2]:
import pandas as pd
import re
import sys
from awsglue.utils import getResolvedOptions
from pyspark.sql import SparkSession
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

spark = SparkSession.builder \
    .config("spark.sql.warehouse.dir", warehouse_path) \
    .config(f"spark.sql.catalog.{catalog_name}", "org.apache.iceberg.spark.SparkCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.warehouse", warehouse_path) \
    .config(f"spark.sql.catalog.{catalog_name}.catalog-impl", "org.apache.iceberg.aws.glue.GlueCatalog") \
    .config(f"spark.sql.catalog.{catalog_name}.io-impl", "org.apache.iceberg.aws.s3.S3FileIO") \
    .config("spark.sql.extensions", "org.apache.iceberg.spark.extensions.IcebergSparkSessionExtensions") \
    .getOrCreate()

In [3]:
sys.argv.append('--JOB_NAME')
sys.argv.append('andres_lagos_notebook_icerberg')
args = getResolvedOptions(sys.argv, ['JOB_NAME'])
glueContext = GlueContext(spark)
job = Job(glueContext)
job.init(args['JOB_NAME'], args)

In [4]:
class Create_Dictionary_Tables():
    "Create the tables of dictionary"
    
    def __init__(self, catalog_name: str, database_name: str) -> None:
        """
        initialize the database parameters
        Arg:
            catalog_name: the glue data catalog name
            database_name: the glye database name of dictionary data
        """
        
        self.database_name = database_name
        self.catalog_name = catalog_name
        self.warehouse_path = f"s3://{bucket_name}/{bucket_prefix}"
    
    def create_column_table(self, column_table_name: str = 'column') -> None:
        """
        check and create the column table
        Arg:
            column_table_name: name of the column's table metadata
        return:
            None
        """
        
        try:
            spark.table(f"{self.catalog_name}.{self.database_name}.{column_table_name}").limit(1).show()
        except:
            spark.sql(f"""
                CREATE TABLE {self.catalog_name}.{self.database_name}.{column_table_name}(
                    column_id string
                    , table_id string
                    , database string
                    , table string
                    , column_name string
                    , description string
                    , data_type string
                    , nullable boolean
                    , is_partitioned boolean
                    , ordinal_position int
                    , is_deleted boolean
                    )
                PARTITIONED BY (table_id)
                LOCATION '{self.warehouse_path}/{column_table_name}'
                TBLPROPERTIES (
                  'table_type'='ICEBERG',
                  'format'='parquet'
                )
                """).show()
            
    def create_table_table(self, table_table_name: str = 'table') -> None:
        """
        check and create the table table
        Arg:
            table_table_name: name of the table's table metadata
        return:
            None
        """
        
        try:
            response = spark.table(f"{self.catalog_name}.{self.database_name}.{table_table_name}").limit(1).show()
        except:
            spark.sql(f"""
            CREATE TABLE {self.catalog_name}.{self.database_name}.{table_table_name}(
                table_id string
                , database string
                , table string
                , description string
                , create_statement string
                , update_crawler string
                , are_columns_quoted boolean
                , classification string
                , columns_ordered string
                , compression_type string
                , delimiter string
                , object_count int
                , record_count int
                , size_key int
                , skip_line int
                , type_data string
                , location string
                , is_view boolean
                , is_deleted boolean
                )
                PARTITIONED BY (table_id)
                LOCATION '{self.warehouse_path}/{table_table_name}'
                TBLPROPERTIES (
                'table_type'='ICEBERG',
                'format'='parquet'
                )""").show()
        
    def create_tables(self) -> None:
        "Execute all of the create table process"
        self.create_column_table()
        self.create_table_table()
        

In [5]:
Create_Dictionary_Tables(catalog_name, database_name).create_tables()

+--------------------+-------------------+---------+---------+--------------------+-----------+---------+--------+--------------+----------------+----------+
|           column_id|           table_id| database|    table|         column_name|description|data_type|nullable|is_partitioned|ordinal_position|is_deleted|
+--------------------+-------------------+---------+---------+--------------------+-----------+---------+--------+--------------+----------------+----------+
|data_lake.austral...|data_lake.australia|data_lake|australia|algorithm-marked-...|       null|  boolean|    true|         false|               1|     false|
+--------------------+-------------------+---------+---------+--------------------+-----------+---------+--------+--------------+----------------+----------+

+-------------------+---------+---------+-----------+--------------------+-----------------+------------------+--------------+---------------+----------------+---------+------------+------------+---------+----

In [6]:
class Land_Metastore():
    """Lands the metadata for schemas listed in the database dictionary.
    
    This function will create the temporary views column_metadata, 
    table_metadata and create_metadata. These tables describe the metadata
    of the tables contained in the database list.
    
    Where a landed value is the string 'None', it represents a null value
    (pandas doesnt support a null types).
    

    Args:
        database_list (list): list of databases
    """
    def __init__(self, database_list: list) -> None:
        "Initialize the process and execute all of the steps"
        
        tables = []
        # for each database determine table list using spark catalog
        for database in database_list:
            table_list = spark.catalog.listTables(database)
            tables += [{'tableName': i.name, 'database': i.database
                        , 'description':i.description, 'tableType':i.tableType
                        , 'isTemporary':i.isTemporary} for i in table_list]

        # we dont want to document temp tales, remove from list.
        tables = [i for i in tables if not i['isTemporary']]
        self.tables = tables

    def get_column_df(self) -> pd.DataFrame:
        "Return pandas databframe of column metadata given list of tables."
       
        all_column_metadata=[]
        for table in self.tables:
            # collect column info
            column_list = spark.catalog.listColumns(table['tableName'], table['database'])
            # can infer ordinal position by incrementing a counter from 1 for each column in list.
            ordinal_position = 1
            for col in column_list: 
                # add each columns metadata to all_column_metadata list
                column_metadata = [table['database'], table['tableName']]+[col[index] for index in range(len(col))]+[ordinal_position]
                ordinal_position+=1
                all_column_metadata.append(column_metadata)

        # set column names
        column_metadata_columns = ['database', 'table','columnName','description', 'dataType','nullable','isPartitioned','isBucket', 'ordinalPosition']
        df_columns = pd.DataFrame(all_column_metadata, columns=column_metadata_columns)
        # resolve missing values
        df_columns.fillna('None', inplace=True)
        return df_columns

    def get_table_df(self) -> pd.DataFrame:
        """Return pandas dataframe of table metadata given list of tables."""
        all_table_metadata = []
        for table in self.tables:
            if table['tableType']!='VIEW':
                table_metadata = list()
                #collect result of describe detail sql command, collect as list and add to metadata list
                table_data = spark.sql(f"DESCRIBE EXTENDED {table['database']}.{table['tableName']}").collect()
                for index in table_data:
                    if index.col_name in ['Database', 'Table', 'Table Properties'
                        , 'Location', 'Storage Properties', 'Partition Provider']:
                        if index.col_name == 'Table Properties':
                            table_metadata.extend([value.split('=')[1] for value in re.split(r",(?!,)",index[1])])
                        else:
                            table_metadata.append(index[1])
                if len(table_metadata) == 19:
                    table_metadata.insert(10, '0')
                    table_metadata.insert(11, '')  
                table_metadata.append(False)
                table_metadata.append(table['description'])
                all_table_metadata.append(table_metadata)
            else:
                table_metadata = [table['database'], table['tableName'], None, None
                  , None, None, '0', None
                  , None, None, None, '0'
                  , '0', '0', '0', None, None
                  , None, None, True, table['description']]
                all_table_metadata.append(table_metadata)
        table_metadata_columns = ['database', 'table', 'crawlerDeserializerV', 'crawlerSerializarV' 
                                , 'updateCrawler', 'areColumnsQuoted', 'avgRecordSize', 'classification' 
                                , 'columnsOrdered', 'compressionType', 'delimiter', 'last_modified_by'
                                ,  'last_modified_time', 'objectCount'
                                , 'recordCount', 'sizeKey', 'skipLine', 'typeData', 'location'
                                , 'storageProperties', 'PartitionProvider', 'is_view', 'description']
        df_tables = pd.DataFrame(all_table_metadata,  columns=table_metadata_columns)
        # drop some columns we dont need
        df_tables.drop(labels = ['crawlerDeserializerV', 'crawlerSerializarV', 'storageProperties'], axis=1, inplace=True)
        # resolve missing values
        df_tables.fillna(value={'avgRecordSize': 0, 'objectCount': 0, 'recordCount': 0, 
            'sizeKey': 0, 'skipLine': 0}, inplace=True)
        df_tables.fillna('None', inplace=True)
        return df_tables


    def get_create_df(self) -> pd.DataFrame:
        """Return pandas databframe of create table statements given list of tables."""
        
        all_create_metadata = []
        for table in self.tables:
            create = spark.sql(f"SHOW CREATE TABLE {table['database']}.{table['tableName']}").collect()[0][0]
            table_metadata = [table['database'], table['tableName']] + [create]
            all_create_metadata.append(table_metadata)
        create_metadata_columns = ['database', 'table','createStatement']
        df_create = pd.DataFrame(all_create_metadata, columns=create_metadata_columns)
        return df_create

    def temp_view_from_pandas(self, df, view_name):
        """Create a temporary view from a pandas dataframe"""

        spark_df = spark.createDataFrame(df)
        spark_df.createOrReplaceTempView(view_name)
        print(f"Temporary table `{view_name}` created.")
        print(f"Row count: {spark_df.count()}")
        
    def main(self) -> None:
        "Execute all of the workflow"
        # collect metadata and land metadata for schema
        metadata_df = self.get_column_df()
        self.temp_view_from_pandas(metadata_df, 'Column_Metadata')
        metadata_df = self.get_table_df()
        self.temp_view_from_pandas(metadata_df, 'Table_Metadata')
        metadata_df = self.get_create_df()
        self.temp_view_from_pandas(metadata_df, 'Create_Metadata')

In [7]:
Land_Metastore(['data_lake']).main()

Temporary table `Column_Metadata` created.
Row count: 54
Temporary table `Table_Metadata` created.
Row count: 4
Temporary table `Create_Metadata` created.
Row count: 4


In [11]:
class Upsert_Metadata():
    "Upsert metadata tables"
    
    def __init__(self, catalog_name: str, database_name: str) -> None:
        """
        initialize the database parameters
        Arg:
            catalog_name: the glue data catalog name
            database_name: the glue database name of dictionary data
        """
        
        self.database_name = database_name
        self.catalog_name = catalog_name
        self.warehouse_path = f"s3://{bucket_name}/{bucket_prefix}"
        
    def upsert_column_data(self, column_table_name: str = 'column') -> None:
        """
        Upsert dictionary table column with all of the available data
        Arg:
            column_table_name: the name of the dictionary column's table
        Return:
            None
        """
        
        spark.sql(f"""
            MERGE INTO {catalog_name}.{database_name}.{column_table_name}  AS glue
                USING Column_Metadata AS temp
                ON glue.database = temp.database
                AND glue.table = temp.table 
                AND glue.column_name = temp.ColumnName
                WHEN MATCHED THEN UPDATE SET
                glue.description=CASE WHEN temp.description = 'None' THEN NULL ELSE temp.description END,
                glue.ordinal_position=temp.ordinalPosition,
                glue.data_type=temp.dataType,
                glue.nullable=temp.nullable,
                glue.is_partitioned=temp.isPartitioned,
                glue.is_deleted=False
            WHEN NOT MATCHED THEN INSERT
            (column_id, table_id, database, table, column_name
            , description, ordinal_position, data_type, nullable,
            is_partitioned, is_deleted)
            VALUES
                (CONCAT_WS('.', temp.database, temp.table, temp.columnName),
                CONCAT_WS('.', temp.database, temp.Table),
                temp.database,
                temp.table,
                temp.columnName,
                CASE WHEN temp.description = 'None' THEN NULL ELSE temp.description END,
                temp.ordinalPosition,
                temp.dataType,
                temp.nullable,
                temp.isPartitioned,
                False
                )
            """).show()
        
    def update_deleted_column(self, column_table_name: str = 'column') -> None:
        """
        update delete column in the column's dictionary table
        Arg:
            column_table_name: the name of the dictionary column's table
        Return:
            None
        """

        spark.sql(f"""CREATE OR REPLACE TEMP VIEW deleted_values AS
            SELECT glue.*
            FROM {catalog_name}.{database_name}.{column_table_name} glue
            ANTI JOIN Column_Metadata temp
                ON glue.database = temp.database
                AND glue.column_name = temp.columnName
                AND glue.table = temp.table
            WHERE glue.database IN (SELECT database FROM Column_Metadata)
            """).show()
        spark.sql(f"""
            MERGE INTO {catalog_name}.{database_name}.{column_table_name} glue
            USING deleted_values deleted
            ON glue.database = deleted.database
                AND glue.column_name = deleted.column_name
                AND glue.table = deleted.table
            WHEN MATCHED THEN UPDATE SET
            glue.is_deleted=True
            """).show()
            
    def create_table_metadata(self) -> None:
        """
        Create table's metadata
        Arg:
            column_table_name: the name of the dictionary table
        Return:
            None
        """

        spark.sql("""
        CREATE OR REPLACE TEMP VIEW stage_table_metadata AS
        SELECT 
            table.database || '.' || table.table AS table_id,
            table.database AS database,
            table.table AS table,
            CASE WHEN table.description ='None' THEN null ELSE table.description END AS description,
            metadata.createStatement as create_statement,
            table.updateCrawler as update_crawler,
            CASE WHEN table.areColumnsQuoted IN ('None', 'none') THEN false ELSE cast(table.areColumnsQuoted as boolean) END AS are_columns_quoted,
            CASE WHEN table.classification IN ('None', 'none') THEN null ELSE table.classification END AS classification,
            CASE WHEN table.columnsOrdered IN ('None', 'none') THEN false ELSE cast(table.columnsOrdered AS boolean) END AS columns_ordered,
            CASE WHEN table.compressionType IN ('None', 'none') THEN null ELSE table.compressionType END AS compression_type,
            CASE WHEN table.delimiter ='None' THEN null ELSE table.delimiter END AS delimiter,
            cast(table.objectCount AS integer) AS object_count,
            cast(table.recordCount AS integer) AS record_count,
            cast(table.sizeKey AS integer) AS size_key,
            cast(table.skipLine AS integer) AS skip_line,
            table.typeData AS type_data,
            table.location,
            is_view,
            False AS is_deleted
        FROM table_metadata table
        JOIN create_metadata metadata
            ON table.database = metadata.database
            AND table.table = metadata.table
        """).show()
            
    def upsert_table_data(self, table_table_name: str = 'table') -> None:
        """
        Upsert dictionary table table with all of the available data
        Arg:
            column_table_table: the name of the dictionary table's table
        Return:
            None
        """

        spark.sql(f"""
        MERGE INTO {catalog_name}.{database_name}.{table_table_name} glue 
        USING stage_table_metadata  temp
            ON glue.database = temp.database
            AND glue.table = temp.table 
        WHEN MATCHED THEN UPDATE SET
            glue.create_statement = temp.create_statement,
            glue.update_crawler = temp.update_crawler,
            glue.are_columns_quoted = temp.are_columns_quoted,
            glue.classification = temp.classification,
            glue.columns_ordered = temp.columns_ordered,
            glue.compression_type = temp.compression_type,
            glue.delimiter = temp.delimiter,
            glue.object_count = temp.object_count,
            glue.record_count = temp.record_count,
            glue.size_key = temp.size_key,
            glue.type_data = temp.type_data,
            glue.location = temp.location,
            glue.is_view = temp.is_view,
            glue.is_deleted = temp.is_deleted
        WHEN NOT MATCHED THEN INSERT *
        """).show()
            
    def update_delete_tables(self, table_table_name: str = 'table') -> None:
        """
        Get delete tables
        Arg:
            table_table_table: the name of the dictionary table's table
        Return:
            None
        """

        spark.sql(f"""
        CREATE OR REPLACE TEMP VIEW deleted_values AS
        SELECT table.*
        FROM {catalog_name}.{database_name}.{table_table_name} table
        ANTI JOIN table_metadata temp
            ON table.database = temp.database
            AND table.table = temp.table
        """).show()
        spark.sql(f"""
        MERGE INTO {catalog_name}.{database_name}.{table_table_name} table
        USING deleted_values temp
        ON table.database = temp.database
            AND table.table = temp.table
        WHEN MATCHED THEN UPDATE SET
            table.is_deleted=True
        """).show()
    
    def upsert_tables(self) -> None:
        "Execute the upsert process"
        self.upsert_column_data()
        self.update_deleted_column()
        self.create_table_metadata()
        self.upsert_table_data()
        self.update_delete_tables()

In [ ]:
Upsert_Metadata(catalog_name, database_name).upsert_tables()

## Reorder Tables

In [79]:
import boto3
from time import sleep

class Update_Metadata:
    """
    Update the metadata tables and add some descriptions
    """
    
    def __init__(self, catalog: str, dictionary_database: str, database: str, output_file: str) -> None:
        """
        initialize the application
        Arg:
            catalog: the glue catalog name
            dictionary_database: name of the quality database
            database: the database that are going to the analyze
            path_athena: the path when all of the athena output is going to be "S3 path"
        return: None
        """
        self.catalog = catalog
        self.dictionary_database = dictionary_database
        self.database = database
        self.athena_client = boto3.client(service_name='athena')
        self.output_file= output_file
        
    def reorder_columns_alphabetically(self, dictionary: str = 'column') -> None:
        """
        reorder column table
        Arg:
            dictionary: the name of the column
        return: None
        """
        spark.sql(f"""CREATE OR REPLACE TEMP VIEW reorder_columns AS
            SELECT table_id,
            column_name, 
            ROW_NUMBER() OVER (PARTITION BY table_id ORDER BY column_name) AS new_ordinal
            FROM {self.catalog}.{self.dictionary_database}.{dictionary} 
            WHERE LOWER(database) = '{self.database.lower()}'
            """)
        spark.sql(f"""MERGE INTO {self.catalog}.{self.dictionary_database}.{dictionary} old
            USING reorder_columns AS new
            ON new.column_name = old.column_name
            AND new.table_id = old.table_id
            WHEN MATCHED THEN UPDATE SET
            old.ordinal_position = new.new_ordinal""")
        spark.sql("DROP VIEW reorder_columns")
        
    def update_database(self) -> None:
        "collect list of tables in database that are not temporary"
        tables = [i['tableName'] for i in spark.sql(f'SHOW TABLES FROM {self.database}').collect() if not i['isTemporary']]
        print(f'Updating tables: {tables} \n')
        for table in tables:
            self.update_table(table=table)
            
    def execute_query(self, query: str, output_file: str = None) -> None:
        """
        Execute the query in Athena
        Arg:
            query: query to execute
            output_file: path of the S3 output
        """
        if output_file is None: output_file = self.output_file
        query_id = athena_client.start_query_execution(QueryString=query.replace('glue_catalog.','')
            , WorkGroup='primary',ResultConfiguration={'OutputLocation': output_file})['QueryExecutionId']
        while True:
            status = athena_client.get_query_execution(QueryExecutionId=query_id)['QueryExecution']['Status']['State']
            if status != 'RUNNING': break 
            sleep(5)
            
    def update_table(self, table: str, dictionary : str = 'column') -> None:
        """
        Use the data dictionary to determine the ordinal position and comment associated
        with each column in a given table. Generate an alter table statement that applies
        these changes to the table.
        ########################################
        """
      # check if table is view, skip if so.
        if self.is_view(table):
            print(f'Cannot update {self.database}.{table} because its a view. \n')
            return

        df_column = spark.sql(f"""SELECT * FROM {self.catalog}.{self.dictionary_database}.{dictionary} 
            WHERE is_deleted = False
            AND LOWER(database) = '{self.database.lower()}'
            AND LOWER(table) = '{table.lower()}'
            ORDER BY ordinal_position, column_name""").toPandas()

        alter = []
        for cn, dt,d in zip(df_column['column_name'], df_column['data_type'], df_column['description']):
            if d:
            # if description include comment
                alter+= [f"{cn} {dt} COMMENT '{d}'"]
            else:
            # else dont include comment
                alter+= [f"`{cn}` {dt}"]
        alter = ', '.join(alter)

        query = f"ALTER TABLE {self.catalog}.{self.database}.{table} REPLACE COLUMNS ({alter})"

        print(query)
        self.execute_query(query)


    def is_view(self, table, dictionary = 'table'):
        """Check if a table exists, if so return True, if view return False."""

        df = spark.sql(f"""SELECT * 
            FROM {self.catalog}.{self.dictionary_database}.{dictionary}
            WHERE LOWER(table) = '{table.lower()}'
            AND LOWER (database) = '{self.database.lower()}'
            AND is_deleted=False""").toPandas()
        if df.shape[0]==0:
            raise Exception(f'Table {table} does not exist in dictionary')

        return df['is_view'][0]


In [80]:
Update_Metadata(catalog='glue_catalog', dictionary_database='dictionary_quality'
    , database='data_lake', output_file='s3://andres-lagos-bucket/query-athena/').update_database()

Updating tables: ['australia', 'kingdom', 'usa', 'zealand'] 

ALTER TABLE glue_catalog.data_lake.australia REPLACE COLUMNS (`algorithm-marked-outlier` boolean, `comments` bigint, `event-id` bigint, `individual-local-identifier` string, `individual-taxon-canonical-name` string, `location-lat` double, `location-long` double, `manually-marked-outlier` string, `sensor-type` string, `study-name` string, `tag-local-identifier` string, `timestamp` string, `visible` boolean)
ALTER TABLE glue_catalog.data_lake.kingdom REPLACE COLUMNS (`algorithm-marked-outlier` boolean, `event-id` bigint, `ground-speed` double, `height-above-ellipsoid` double, `individual-local-identifier` string, `individual-taxon-canonical-name` string, `location-lat` double, `location-long` double, `manually-marked-outlier` boolean, `sensor-type` string, `study-name` string, `tag-local-identifier` string, `timestamp` string, `visible` boolean)
ALTER TABLE glue_catalog.data_lake.usa REPLACE COLUMNS (`algorithm-marked-outlier`

In [27]:
#catalog and dictionary_database
Update_Metadata(catalog='glue_catalog', dictionary_database='dictionary_quality'
    , database='data_lake', output_file='s3://andres-lagos-bucket/query-athena/').reorder_columns_alphabetically()

In [59]:
athena_client.get_query_execution(QueryExecutionId=query_id)['QueryExecution']['Status']['State']

'RUNNING'


## Data Quality test

In [13]:
from pyspark.sql import SparkSession, utils

In [15]:
try:
    spark.sql("""ALTER TABLE glue_catalog.dictionary_quality.column ADD COLUMNS null_test BOOLEAN, length_test INT""")
    spark.sql("""ALTER TABLE glue_catalog.dictionary_quality.table ADD COLUMNS Key STRING AFTER description""")
except utils.AnalysisException:
    print("Column already exits")

Column already exits


In [8]:
spark.sql("""UPDATE glue_catalog.dictionary_quality.table
    SET key = 'uuid "fake"'
    WHERE table IN ('zealand', 'australia', 'usa', 'kingdom')""")


DataFrame[]


In [32]:
spark.sql("""SELECT * FROM glue_catalog.dictionary_quality.column""")

DataFrame[column_id: string, table_id: string, database: string, table: string, column_name: string, description: string, data_type: string, nullable: boolean, is_partitioned: boolean, ordinal_position: int, is_deleted: boolean, length_test: int, null_test: boolean]


In [53]:
spark.sql("""
UPDATE glue_catalog.dictionary_quality.column
SET null_test = True
WHERE column_name IN ('visible','event-id')
""").collect()

[]


In [56]:
spark.sql("""
UPDATE glue_catalog.dictionary_quality.column
SET length_test = 3
WHERE column_name = 'sensor-type'
""").collect()

[]


In [60]:
spark.sql("""SELECT database, table, column_name, length_test, 
        CONCAT(column_id,'.','is_not_length_', length_test) AS test_name
    FROM glue_catalog.dictionary_quality.column 
    where length_test IS NOT null AND is_deleted=false""")
df_null_test.createOrReplaceTempView("length_test")
spark.sql("SELECT * FROM length_test").show()

+---------+---------+-----------+--------------------+
| database|    table|column_name|           test_name|
+---------+---------+-----------+--------------------+
|data_lake|australia|   event-id|data_lake.austral...|
|data_lake|australia|    visible|data_lake.austral...|
|data_lake|  kingdom|    visible|data_lake.kingdom...|
|data_lake|  kingdom|   event-id|data_lake.kingdom...|
|data_lake|      usa|   event-id|data_lake.usa.eve...|
|data_lake|      usa|    visible|data_lake.usa.vis...|
|data_lake|  zealand|   event-id|data_lake.zealand...|
|data_lake|  zealand|    visible|data_lake.zealand...|
+---------+---------+-----------+--------------------+


In [59]:
df_null_test = spark.sql("""SELECT database, table, column_name
        , CONCAT_WS('.',column_id,'is_null') AS test_name
    FROM glue_catalog.dictionary_quality.column
    WHERE null_test=true
    AND is_deleted=false""")
df_null_test.createOrReplaceTempView("null_test")
spark.sql("SELECT * FROM null_test").show()

+---------+---------+-----------+--------------------+
| database|    table|column_name|           test_name|
+---------+---------+-----------+--------------------+
|data_lake|australia|   event-id|data_lake.austral...|
|data_lake|australia|    visible|data_lake.austral...|
|data_lake|  kingdom|    visible|data_lake.kingdom...|
|data_lake|  kingdom|   event-id|data_lake.kingdom...|
|data_lake|      usa|   event-id|data_lake.usa.eve...|
|data_lake|      usa|    visible|data_lake.usa.vis...|
|data_lake|  zealand|   event-id|data_lake.zealand...|
|data_lake|  zealand|    visible|data_lake.zealand...|
+---------+---------+-----------+--------------------+
